Здравствуйте!

Для имплементации выбрал статью: Атака на VAD с помощью сгенерированной музыки: https://vk.cc/arC7rT

В ней авторы организуют adversarial атаку на голосового помощника -- добавляют музыку, которая мешает помошнику реагировать на команды начала работы (wake-words)


Идея метода в том, чтобы обучить дифференцируемый алгоритм для генерации музыки (Karplus-Strong) градиентным спуском, максимизируя лосс для wake-word detection

Плюсы метода, в отличие от аналогов:  

1) Он не white-box, у нас может не быть доступа к атакуемой модели, главное, чтобы мы могли ±эмулировать градиенты, протекающие через неё (что и сделали авторы, натренировав модель, архитектурно схожую с голосовым помошником Amazon Alexa).

2) Можно натренировать на любые команды

3) Показывает хорошие результаты "в бою" -- при добавление сгенерированной музыки к речи в реальном мире

Сначала я написал автору статьи (Juncheng B. Li) письмо, и попросил выслать мне веса wake-word detection модели, но, к сожалению, у него больше нет к ним доступа, т.к. он уволился из компании, где проводил исследование (Bosh). Он посоветовал мне использовать open-source модели для keyword-spotting в качестве бейзлайна, что я и сделал

Взял самый простой, на мой взгляд, проект на гитхабе для keyword-spotting https://github.com/castorini/honk 

Пусть wake-word будет слово "Yes", организую атаку на него. Dataset -- Google Speech Commands Dataset https://ai.googleblog.com/2017/08/launching-speech-commands-dataset.html

Посмотрим, как работает бейзлайн

In [10]:
!python -W ignore -m main --type eval --wanted_words yes --n_labels 12 --model res8-narrow --input_file honk/model/res8_narrow.pt --no_cuda --data_folder honk/data/speech_dataset

Test accuracy: 0.82, test f1-score: 0.88


Неплохо!

Пример речи

In [11]:
import numpy as np
from IPython.display import Audio
rate = 16000
speech_example = np.load("audio_example.npy")

In [12]:
Audio(speech_example, rate=rate)

Реализовал KS алгоритм на PyTorch. Посмотрим, что будет, если добавить генерируемую им музыку к речи.

In [13]:
!python -W ignore -m main --use_noise True --type eval --wanted_words yes --n_labels 12 --model res8-narrow --input_file honk/model/res8_narrow.pt --no_cuda --data_folder honk/data/speech_dataset

Test accuracy: 0.80, test f1-score: 0.86


Метрики немного снизились

Послушаем

In [14]:
from KS_Strong.model import create_sound
string = create_sound()
Audio(speech_example + string, rate=rate)

У меня нет своего GPU, поэтому я арендовал машину на сайте https://immers.cloud/ . RTX2080TI 16 Gb ОЗУ. Развернул там окружение и всё такое

Одной из проблем было то, что MFCC алгоритм, который используется для preprocessing, в бейзлайне был взять из librosa audio. Чтобы градиенты протекали сквозь бейзлайн, заменил на версию из torchaudio

Кратко, что делаем: генерируем звук с помощью Karplus-Strong, добавляем его к аудиозаписям слов, максимизируем CrossEntropy Loss, добавляя знак минус (минимизируем минус лосс == максимизируем лосс)

Запустим тренировку (последние веса я закоммитил в папку weights, и следующую ячейку можно пропустить):

In [15]:
# !bash run_train.sh

Логи лежать в папке runs. Если установлен tb-nightly, логи можно смотреть в ноутбуке, иначен нужно запустить tensorboard отдельно 

In [16]:
%load_ext tensorboard
%tensorboard --logdir "./runs"

Reusing TensorBoard on port 6007 (pid 48862), started 21:59:02 ago. (Use '!kill 48862' to kill it.)

Оценим качество распознавания с натренированным алгоритмом

In [17]:
!python -W ignore -m main --use_noise True --path_noiser weights/epoch_14.pth True --type eval --wanted_words yes  --dev_every 1 --n_labels 12 --model res8-narrow --input_file honk/model/res8_narrow.pt --no_cuda --data_folder honk/data/speech_dataset

Test accuracy: 0.48, test f1-score: 0.55


Сильно упали точность и F-мера -- атака удалась

Пример звуков

In [18]:
string = create_sound("./weights/epoch_14.pth")
Audio(speech_example + string, rate=rate)

Adversarial example не представляет труда для распознания человеком (субъективно), но сильно снижает качество модели


Звуки похожи на результаты авторов статьи


https://youtu.be/r4XXGDVs0f8?t=42

Атака работает, этот факт можно использовать для защиты wake-word detection моделей, делая их более робастными к такого типа атакам

Как можно улучшить результат? Дольше обучать, перебрать гиперпараметры и т.п. стандартные подходы, которые работают при наличии ресурсов